In [30]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import ast


In [31]:
#import excel from this path: "C:\Users\nicholas.t.norris\OneDrive - Vail Resorts Management Company\Documents\Python\vr-mkt-share-automation\240814 TAM Master V1 -ED.xlsx"
file_path = r"C:\Users\nicholas.t.norris\OneDrive - Vail Resorts Management Company\Documents\Python\vr-mkt-share-automation\240814 TAM Master V1 -ED.xlsx"
# Read the Excel file
mkt_raw = pd.read_excel(file_path, sheet_name='US 12 yr - by CBSA')

vri_raw = pd.read_excel(file_path, sheet_name='VRI data dump', index_col=0)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\nicholas.t.norris\\OneDrive - Vail Resorts Management Company\\Documents\\Python\\vr-mkt-share-automation\\240814 TAM Master V1 -ED.xlsx'

In [32]:
#only keep the 'Season', 'Ticket_2223_2324', 'Ticket type vVRI', 'Visitation Geo', 'ToRegion (VRI)', 'Visitation', columns from mkt
mkt = mkt_raw[['Season', 'Ticket_2223_2324', 'Ticket type vVRI', 'Visitation Geo', 'ToRegion (VRI)', 'Visitation']]
#in mkt, rename 'Ticket_2223_2324' to 'Ticket LOB', 'Ticket type vVRI' to 'Ticket Type', 'Visitation Geo' to 'VisGeo', 'ToRegion (VRI)' to 'Region', and 'Visitation' to 'SV'
mkt = mkt.rename(columns={
    'Ticket_2223_2324': 'Ticket LOB',
    'Ticket type vVRI': 'Ticket Type',
    'Visitation Geo': 'VisGeo',
    'ToRegion (VRI)': 'Region',
    'Visitation': 'SV'
})
#keep every column except 'Resort State' from vri_raw
vri = vri_raw.loc[:, vri_raw.columns != 'Resort State']
#in vri, rename 'Sub Region' to 'Region', 'Ticket Type Grouped' to 'Ticket Type', 'Line of Business (incl. EDP)' to 'LOB', 'Season Year' to 'Season', and 'SVs' to 'SV'
vri = vri.rename(columns={ 
    'Sub Region (adj.)': 'Region',
    'Ticket Type Grouped': 'Ticket Type',
    'Line Of Business  (incl. EDP)': 'LOB',
    'Season Year': 'Season',
    'SVs': 'SV',
    'Visitation Geo': 'VisGeo'
})
#replace all instances of (VRI) in the 'Ticket Type' column with '17MTN'
vri['VRI or Peak'] = vri['VRI or Peak'].str.replace('VRI', '17MTN', regex=False)


In [33]:
print(mkt)

        Season                        Ticket LOB Ticket Type         VisGeo  \
0      2023/24                       Season pass        Pass  International   
1      2023/24                       Paid ticket        Paid  International   
2      2023/24  Other (e.g. comp, special offer)        Paid  International   
3      2023/24                 Frequency product        Pass  International   
4      2023/24                       Paid ticket        Paid  International   
...        ...                               ...         ...            ...   
84653  2021/22                               NaN        Paid    Destination   
84654  2020/21                               NaN        Paid    Destination   
84655  2019/20                               NaN        Paid    Destination   
84656  2018/19                               NaN        Paid    Destination   
84657  2014/15                               NaN        Paid    Destination   

                  Region     SV  
0              No

In [34]:
print(vri)

                Region    Sub Region VRI or Peak            Resort  \
NaN       Mid Atlantic  Mid Atlantic          7S  Hidden Valley PA   
NaN       Mid Atlantic  Mid Atlantic          7S  Hidden Valley PA   
NaN       Mid Atlantic  Mid Atlantic          7S  Hidden Valley PA   
NaN       Mid Atlantic  Mid Atlantic          7S  Hidden Valley PA   
NaN       Mid Atlantic  Mid Atlantic          7S  Hidden Valley PA   
..                 ...           ...         ...               ...   
NaN  Pacific Northwest       WestxWB       17MTN      Stevens Pass   
NaN  Pacific Northwest       WestxWB       17MTN      Stevens Pass   
NaN  Pacific Northwest       WestxWB       17MTN      Stevens Pass   
NaN  Pacific Northwest       WestxWB       17MTN      Stevens Pass   
NaN  Pacific Northwest       WestxWB       17MTN      Stevens Pass   

    Ticket Type               LOB         VisGeo   Season         SV  \
NaN        Paid              Paid    Destination  2022/23        NaN   
NaN        Paid

### Do i need to remove comp, if so, how?

In [35]:
#use groupby to sum the 'Visitation' column in mkt by 'Season' where 'Ticket_2223_2324' is not 'Other (e.g. comp, special offer) and transpose the result
#mkt_total = mkt[mkt['Ticket_2223_2324'] != 'Other (e.g. comp, special offer)'].groupby('Season')['Visitation'].sum().reset_index()


# Define the function which does the calculations

In [36]:
def generate_table(
    mkt_df,
    vri_df,
    filters=None,
    group_column='VRI or Peak',
    total_label='Total industry visits',
    metric='SV',
    group_order=['17MTN', 'Peak', '7S'],
    rename_index_map=None
):
    """
    Generate a summary DataFrame with optional filters and share calculations.

    Parameters:
    - mkt_df, vri_df: DataFrames
    - filters: dict of {column_name: filter_value or None}
    - group_column: column in vri_df to group by (e.g. 'VRI or Peak')
    - total_label: row label for market total
    - metric: column to aggregate
    - group_order: list of group values to order rows
    - rename_index_map: optional dict to rename index labels

    Returns:
    - DataFrame with total visits and share calculations
    """

    def apply_filters(df, filters):
        if not filters:
            return df
        for col, val in filters.items():
            if val is not None:
                df = df[df[col] == val]
        return df

    # Apply filters
    mkt_subset = apply_filters(mkt_df, filters)
    vri_subset = apply_filters(vri_df, filters)

    # Market total
    mkt_total = mkt_subset.groupby('Season')[metric].sum().reset_index()
    mkt_total = mkt_total.set_index('Season').T
    mkt_total.index = [total_label]

    # VRI group totals
    vri_grouped = vri_subset.groupby(['Season', group_column])[metric].sum().reset_index()
    vri_pivot = vri_grouped.pivot(index=group_column, columns='Season', values=metric)

    # Reorder and rename
    vri_pivot = vri_pivot.reindex(group_order)
    if rename_index_map:
        vri_pivot.index = [rename_index_map.get(i, i) for i in vri_pivot.index]

    # Combine and compute shares
    combined = pd.concat([mkt_total, vri_pivot], axis=0)
    for idx in vri_pivot.index:
        share_label = idx.replace('Visits', 'Share')
        combined.loc[share_label] = combined.loc[idx] / combined.loc[total_label]
    combined = combined.fillna(0).round(2)

    return combined


In [37]:
rename_map = {
    '17MTN': 'VRI Visits (17MTN)',
    'Peak': 'VRI Visits (Peak)',
    '7S': 'VRI Visits (7S)'
}

# Everything below this uses the same block of code, but just adjusts the filters. Use these

## Ticket Type

In [38]:
regions = [
    None
]

ticket_types = ['Pass', 'Paid']
vis_geo_types = [None]

tt = {}

for region in regions:
    for ticket_type in ticket_types:
        for visgeo in vis_geo_types:
            label = f"{region} - {ticket_type} - {visgeo}"
            print(f"Generating table for: {label}")

            try:
                table = generate_table(
                    mkt_df=mkt,
                    vri_df=vri,
                    filters={
                        'Region': region,
                        'Ticket Type': ticket_type,
                        'VisGeo': visgeo
                    },
                    group_column='VRI or Peak',
                    total_label='Total industry visits',
                    rename_index_map=rename_map
                )
                tt[label] = table
            except Exception as e:
                print(f"⚠️ Failed for {label}: {e}")


Generating table for: None - Pass - None
Generating table for: None - Paid - None


## Region

In [39]:
regions = [
    'Rocky Mountain', 'Midwest', 'Mid Atlantic',
    'Northeast', 'Pacific Southwest', 'Pacific Northwest'
]

ticket_types = [None]
vis_geo_types = [None]

reg = {}

for region in regions:
    for ticket_type in ticket_types:
        for visgeo in vis_geo_types:
            label = f"{region} - {ticket_type} - {visgeo}"
            print(f"Generating table for: {label}")

            try:
                table = generate_table(
                    mkt_df=mkt,
                    vri_df=vri,
                    filters={
                        'Region': region,
                        'Ticket Type': ticket_type,
                        'VisGeo': visgeo
                    },
                    group_column='VRI or Peak',
                    total_label='Total industry visits',
                    rename_index_map=rename_map
                )
                reg[label] = table
            except Exception as e:
                print(f"⚠️ Failed for {label}: {e}")


Generating table for: Rocky Mountain - None - None
Generating table for: Midwest - None - None
Generating table for: Mid Atlantic - None - None
Generating table for: Northeast - None - None
Generating table for: Pacific Southwest - None - None
Generating table for: Pacific Northwest - None - None


## VisGeo

In [40]:
regions = [
    None
]

ticket_types = [None]
vis_geo_types = ['Local', 'Destination', 'International']

vg = {}

for region in regions:
    for ticket_type in ticket_types:
        for visgeo in vis_geo_types:
            label = f"{region} - {ticket_type} - {visgeo}"
            print(f"Generating table for: {label}")

            try:
                table = generate_table(
                    mkt_df=mkt,
                    vri_df=vri,
                    filters={
                        'Region': region,
                        'Ticket Type': ticket_type,
                        'VisGeo': visgeo
                    },
                    group_column='VRI or Peak',
                    total_label='Total industry visits',
                    rename_index_map=rename_map
                )
                vg[label] = table
            except Exception as e:
                print(f"⚠️ Failed for {label}: {e}")


Generating table for: None - None - Local
Generating table for: None - None - Destination
Generating table for: None - None - International


## Region x Ticket Type

In [41]:
regions = [
    'Rocky Mountain', 'Midwest', 'Mid Atlantic',
    'Northeast', 'Pacific Southwest', 'Pacific Northwest'
]

ticket_types = ['Pass', 'Paid']
vis_geo_types = [None]

reg_tt = {}

for region in regions:
    for ticket_type in ticket_types:
        for visgeo in vis_geo_types:
            label = f"{region} - {ticket_type} - {visgeo}"
            print(f"Generating table for: {label}")

            try:
                table = generate_table(
                    mkt_df=mkt,
                    vri_df=vri,
                    filters={
                        'Region': region,
                        'Ticket Type': ticket_type,
                        'VisGeo': visgeo
                    },
                    group_column='VRI or Peak',
                    total_label='Total industry visits',
                    rename_index_map=rename_map
                )
                reg_tt[label] = table
            except Exception as e:
                print(f"⚠️ Failed for {label}: {e}")


#reorder the keys in reg_tt to go through regions first, then ticket types, then visgeo
new_order = ['Rocky Mountain - Pass - None',
             'Midwest - Pass - None',
             'Mid Atlantic - Pass - None',
             'Northeast - Pass - None',
             'Pacific Southwest - Pass - None',
             'Pacific Northwest - Pass - None',
             'Rocky Mountain - Paid - None',
             'Midwest - Paid - None',
             'Mid Atlantic - Paid - None',
             'Northeast - Paid - None',
             'Pacific Southwest - Paid - None',
             'Pacific Northwest - Paid - None']
reg_tt = {key: reg_tt[key] for key in new_order if key in reg_tt}

print("\n new order: \n")
#print the keys in reg_tt
for key in reg_tt:
    print(key)


Generating table for: Rocky Mountain - Pass - None
Generating table for: Rocky Mountain - Paid - None
Generating table for: Midwest - Pass - None
Generating table for: Midwest - Paid - None
Generating table for: Mid Atlantic - Pass - None
Generating table for: Mid Atlantic - Paid - None
Generating table for: Northeast - Pass - None
Generating table for: Northeast - Paid - None
Generating table for: Pacific Southwest - Pass - None
Generating table for: Pacific Southwest - Paid - None
Generating table for: Pacific Northwest - Pass - None
Generating table for: Pacific Northwest - Paid - None

 new order: 

Rocky Mountain - Pass - None
Midwest - Pass - None
Mid Atlantic - Pass - None
Northeast - Pass - None
Pacific Southwest - Pass - None
Pacific Northwest - Pass - None
Rocky Mountain - Paid - None
Midwest - Paid - None
Mid Atlantic - Paid - None
Northeast - Paid - None
Pacific Southwest - Paid - None
Pacific Northwest - Paid - None


## Region x VisGeo

In [42]:
regions = [
    'Rocky Mountain', 'Midwest', 'Mid Atlantic',
    'Northeast', 'Pacific Southwest', 'Pacific Northwest'
]

ticket_types = [None]
vis_geo_types = ['Local', 'Destination', 'International']

reg_vg = {}

for region in regions:
    for ticket_type in ticket_types:
        for visgeo in vis_geo_types:
            label = f"{region} - {ticket_type} - {visgeo}"
            print(f"Generating table for: {label}")

            try:
                table = generate_table(
                    mkt_df=mkt,
                    vri_df=vri,
                    filters={
                        'Region': region,
                        'Ticket Type': ticket_type,
                        'VisGeo': visgeo
                    },
                    group_column='VRI or Peak',
                    total_label='Total industry visits',
                    rename_index_map=rename_map
                )
                reg_vg[label] = table
            except Exception as e:
                print(f"⚠️ Failed for {label}: {e}")

#reorder the keys in reg_vg to go through regions first, then ticket types, then visgeo
new_order = ['Rocky Mountain - None - Local', 
             'Midwest - None - Local',
             'Mid Atlantic - None - Local',
             'Northeast - None - Local',
             'Pacific Southwest - None - Local',
             'Pacific Northwest - None - Local',
             'Rocky Mountain - None - Destination',
             'Midwest - None - Destination',
             'Mid Atlantic - None - Destination',
             'Northeast - None - Destination',
             'Pacific Southwest - None - Destination',
             'Pacific Northwest - None - Destination',
             'Rocky Mountain - None - International',
             'Midwest - None - International',
             'Mid Atlantic - None - International',
             'Northeast - None - International',
             'Pacific Southwest - None - International',
             'Pacific Northwest - None - International']
reg_vg = {key: reg_vg[key] for key in new_order if key in reg_vg}

print("\n new order: \n")
#print the keys in reg_vg
for key in reg_vg:
    print(key)


Generating table for: Rocky Mountain - None - Local
Generating table for: Rocky Mountain - None - Destination
Generating table for: Rocky Mountain - None - International
Generating table for: Midwest - None - Local
Generating table for: Midwest - None - Destination
Generating table for: Midwest - None - International
Generating table for: Mid Atlantic - None - Local
Generating table for: Mid Atlantic - None - Destination
Generating table for: Mid Atlantic - None - International
Generating table for: Northeast - None - Local
Generating table for: Northeast - None - Destination
Generating table for: Northeast - None - International
Generating table for: Pacific Southwest - None - Local
Generating table for: Pacific Southwest - None - Destination
Generating table for: Pacific Southwest - None - International
Generating table for: Pacific Northwest - None - Local
Generating table for: Pacific Northwest - None - Destination
Generating table for: Pacific Northwest - None - International

 ne

## Ticket Type x VisGeo

In [43]:
regions = [
    None
]

ticket_types = ['Pass', 'Paid']
vis_geo_types = ['Local', 'Destination', 'International']

tt_vg = {}

for region in regions:
    for ticket_type in ticket_types:
        for visgeo in vis_geo_types:
            label = f"{region} - {ticket_type} - {visgeo}"
            print(f"Generating table for: {label}")

            try:
                table = generate_table(
                    mkt_df=mkt,
                    vri_df=vri,
                    filters={
                        'Region': region,
                        'Ticket Type': ticket_type,
                        'VisGeo': visgeo
                    },
                    group_column='VRI or Peak',
                    total_label='Total industry visits',
                    rename_index_map=rename_map
                )
                tt_vg[label] = table
            except Exception as e:
                print(f"⚠️ Failed for {label}: {e}")


#reorder the keys in tt_vg to go through regions first, then ticket types, then visgeo
new_order = ['None - Pass - Local', 
             'None - Pass - Destination',
             'None - Pass - International',
             'None - Paid - Local',
             'None - Paid - Destination',
             'None - Paid - International']
tt_vg = {key: tt_vg[key] for key in new_order if key in tt_vg}

print("\n new order: \n")
#print the keys in tt_vg
for key in tt_vg:
    print(key)


Generating table for: None - Pass - Local
Generating table for: None - Pass - Destination
Generating table for: None - Pass - International
Generating table for: None - Paid - Local
Generating table for: None - Paid - Destination
Generating table for: None - Paid - International

 new order: 

None - Pass - Local
None - Pass - Destination
None - Pass - International
None - Paid - Local
None - Paid - Destination
None - Paid - International


## Region x Ticket Type x VisGeo

In [44]:
regions = [
    'Rocky Mountain', 'Midwest', 'Mid Atlantic',
    'Northeast', 'Pacific Southwest', 'Pacific Northwest'
]

ticket_types = ['Pass', 'Paid']
vis_geo_types = ['Local', 'Destination', 'International']

reg_tt_vg = {}

for region in regions:
    for ticket_type in ticket_types:
        for visgeo in vis_geo_types:
            label = f"{region} - {ticket_type} - {visgeo}"
            print(f"Generating table for: {label}")

            try:
                table = generate_table(
                    mkt_df=mkt,
                    vri_df=vri,
                    filters={
                        'Region': region,
                        'Ticket Type': ticket_type,
                        'VisGeo': visgeo
                    },
                    group_column='VRI or Peak',
                    total_label='Total industry visits',
                    rename_index_map=rename_map
                )
                reg_tt_vg[label] = table
            except Exception as e:
                print(f"⚠️ Failed for {label}: {e}")


#reorder the keys in tt_vg to go through regions first, then ticket types, then visgeo
new_order = ['Rocky Mountain - Pass - Local',
             'Midwest - Pass - Local',
             'Mid Atlantic - Pass - Local',
             'Northeast - Pass - Local',
             'Pacific Southwest - Pass - Local',
             'Pacific Northwest - Pass - Local',
             'Rocky Mountain - Pass - Destination',
             'Midwest - Pass - Destination',
             'Mid Atlantic - Pass - Destination',
             'Northeast - Pass - Destination',
             'Pacific Southwest - Pass - Destination',
             'Pacific Northwest - Pass - Destination',
             'Rocky Mountain - Pass - International',
             'Midwest - Pass - International',
             'Mid Atlantic - Pass - International',
             'Northeast - Pass - International',
             'Pacific Southwest - Pass - International',
             'Pacific Northwest - Pass - International',
             'Rocky Mountain - Paid - Local',
             'Midwest - Paid - Local',
             'Mid Atlantic - Paid - Local',
             'Northeast - Paid - Local',
             'Pacific Southwest - Paid - Local',
             'Pacific Northwest - Paid - Local',
             'Rocky Mountain - Paid - Destination',
             'Midwest - Paid - Destination',
             'Mid Atlantic - Paid - Destination',
             'Northeast - Paid - Destination',
             'Pacific Southwest - Paid - Destination',
             'Pacific Northwest - Paid - Destination',
             'Rocky Mountain - Paid - International',
             'Midwest - Paid - International',
             'Mid Atlantic - Paid - International',
             'Northeast - Paid - International',
             'Pacific Southwest - Paid - International',
             'Pacific Northwest - Paid - International']
reg_tt_vg = {key: reg_tt_vg[key] for key in new_order if key in reg_tt_vg}

print("\n new order: \n")
#print the keys in reg_tt_vg
for key in reg_tt_vg:
    print(key)


Generating table for: Rocky Mountain - Pass - Local
Generating table for: Rocky Mountain - Pass - Destination
Generating table for: Rocky Mountain - Pass - International
Generating table for: Rocky Mountain - Paid - Local
Generating table for: Rocky Mountain - Paid - Destination
Generating table for: Rocky Mountain - Paid - International
Generating table for: Midwest - Pass - Local
Generating table for: Midwest - Pass - Destination
Generating table for: Midwest - Pass - International
Generating table for: Midwest - Paid - Local
Generating table for: Midwest - Paid - Destination
Generating table for: Midwest - Paid - International
Generating table for: Mid Atlantic - Pass - Local
Generating table for: Mid Atlantic - Pass - Destination
Generating table for: Mid Atlantic - Pass - International
Generating table for: Mid Atlantic - Paid - Local
Generating table for: Mid Atlantic - Paid - Destination
Generating table for: Mid Atlantic - Paid - International
Generating table for: Northeast - 

In [45]:
print(reg_tt_vg["Mid Atlantic - Pass - Local"])

Season                   2012/13      2013/14      2014/15    2015/16  \
Total industry visits 965,832.62 1,055,634.25 1,008,862.20 730,050.95   
VRI Visits (17MTN)          0.00         0.00         0.00       0.00   
VRI Visits (Peak)           0.00         0.00         0.00       0.00   
VRI Visits (7S)             0.00         0.00         0.00       0.00   
VRI Share (17MTN)           0.00         0.00         0.00       0.00   
VRI Share (Peak)            0.00         0.00         0.00       0.00   
VRI Share (7S)              0.00         0.00         0.00       0.00   

Season                   2016/17    2017/18    2018/19    2019/20    2020/21  \
Total industry visits 705,698.23 818,384.04 625,713.47 500,751.95 926,646.96   
VRI Visits (17MTN)          0.00       0.00       0.00       0.00       0.00   
VRI Visits (Peak)           0.00       0.00       0.00 177,976.00 197,073.00   
VRI Visits (7S)             0.00       0.00       0.00       0.00       0.00   
VRI Share (17MT

## export to excel

In [33]:
import openpyxl

output_file = r"C:\Users\nicholas.t.norris\OneDrive - Vail Resorts Management Company\Documents\Python\vr-mkt-share-automation\output.xlsx"
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Market Share Data"

row_start = 1403
col_index = 9  # Column I (1-based index)

for label, table in reg_tt_vg.items():
    # Write table values only (no headers, no index)
    for i, (_, row) in enumerate(table.iterrows()):
        for j, value in enumerate(row):
            ws.cell(row=row_start + i, column=col_index + j, value=value)

    # Move to the next table position, always 23 rows down from last starting point
    row_start += 23

# Save the workbook
wb.save(output_file)


Next steps:
- limit amount years (do at beginning of code after reading in data)
- reorder every dictionaries to match excel order
- use the above export to excel code, convert to function, and then call the function for every dictionary specifing the row it will start on